## Project: GenAI for Customer Acquisition

### Load Libraries

In [91]:
# Load the libraries
from openai import OpenAI
import os
import base64
from IPython.display import Image, display, Markdown
import pandas as pd
from io import StringIO

### OpenAI Setup

In [31]:
!pip install openai -q

In [32]:
from google.colab import userdata
api_key = userdata.get('genai_course')

In [33]:
#Define the model
MODEL = "gpt-4o"

In [34]:
#Connect to OpenAi api
client = OpenAI(api_key=api_key)

### Mounting Google Drive and Setting Up the Directory


In this section, we mount Google Drive to access files stored in it and specify the directory containing the PDF files we want to convert.



In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Define the directory containing the PDF files
ROOT_DIR = '/content/drive/MyDrive/GenAI/Capstone 1'

## PDF to Image Converter

### Installing and Importing Required Libraries

In [37]:
# Install PyMuPDF and Pillow libraries
!pip install PyMuPDF Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 44.6 MB/s eta 0:00:00


We import the libraries required for PDF manipulation and image processing.



In [38]:
# Import the fitz module from PyMuPDF for PDF handling
import fitz  # PyMuPDF
# Import the Image module from Pillow for image processing
from PIL import Image
# Import os for operating system dependent functionality
import os

* `fitz` provides functions to read and manipulate PDF files.
* `Image` from `PIL` allows us to create and modify images.
* `os` helps in interacting with the operating system, such as reading files and directories.

### Defining the PDF to JPG Conversion Function

In [39]:
def pdf_to_jpg(directory):
    # Iterate over all files in the specified directory
    for filename in os.listdir(directory):
        # Check if the file is a PDF
        if filename.endswith('.pdf'):
            # Construct the full file path
            pdf_path = os.path.join(directory, filename)
            # Open the PDF document
            pdf_document = fitz.open(pdf_path)
            # Iterate over each page in the PDF
            for page_number in range(len(pdf_document)):
                # Load the page by its index
                page = pdf_document.load_page(page_number)
                # Render the page to a pixmap (an in-memory image)
                pix = page.get_pixmap()

                # Construct the output image file path
                image_path = os.path.join(
                    directory,
                    f"{os.path.splitext(filename)[0]}_page_{page_number + 1}.jpg"
                )
                # Create an image object from the pixmap data
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                # Save the image in JPG format
                img.save(image_path)

    # Print a message when all conversions are done
    print("All PDF files have been converted")

In [47]:
directories = [ "Regatta", "Dim Sum"]
for folder in directories:
    directory = os.path.join(ROOT_DIR,folder)
    pdf_to_jpg(directory)


All PDF files have been converted
All PDF files have been converted


# Defining the System Prompt

 We define a detailed system prompt that instructs the GPT model on how to convert the menu images into a structured Excel format.



In [86]:
# Define the system prompt with detailed instructions
system_prompt = """
Convert the menu images to s structured excel sheet
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subcategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps users fill out each row correctly, following the detailed instructions provided.


Rules:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and Subcategory names are repeated for items under the same subcategory.
- Specific columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Detailed item information including names, prices, descriptions, and more should be unique for each entry.
- The content of the uploaded menu will be appended to the current menu without deleting any of the current menu.

Columns Guide:

| Column | Name                             | Description                                       | Accepted Values                       | Example         |
|--------|----------------------------------|---------------------------------------------------|----------------------------------------|-----------------|
| A      | Category TitlePT                   | Category names in Portuguese                      | Text, 256 characters max               | Bebidas         |
| B      | Category TitleEN        | (Optional) English translations of category titles| Text, 256 characters max               | Beverages       |
| C      | Subcategory TitlePT                | (Optional) Subcategory titles in Portuguese       | Text, 256 characters max or leave blank| Sucos           |
| D      | Subcategory TitleEN     | (Optional) English translations of subcategories  | Text, 256 characters max or leave blank| Juices          |
| E      | Item NamePT                        | Item names in Portuguese                          | Text, 256 characters max               | Água Mineral    |
| F      | Item NameEN             | (Optional) English translations of item names     | Text, 256 characters max or leave blank| Mineral Water   |
| G      | Item Price                       | Price of each item                                | Text                                  | 2.50 or 2,50    |
| H      | Calories                         | (Optional) The caloric content of each item       | Numeric                               | 150             |
| I      | Portion Size                     | Portion size for each item                        | Text                                  | 500ml           |
| J      | Availability                     | (Optional) Current availability of the item       | Numeric: 1 for Yes, 0 for No          | 1               |
| K      | Item DescriptionPT   | (Optional) Description in Portuguese              | Text, 500 characters max              | Contains essential minerals |
| L      | Item DescriptionEN      | (Optional) Description in English                 | Text, 500 characters max              | Contains essential minerals |

Notes:
- If Cannot translate , Copy Protugese category, subcategory  names in English columns
- Ensure all data entered follows the specified formats to maintain the integrity of the database.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""

This prompt provides the model with comprehensive instructions on how to process the menu images and the exact format expected for the Excel output. It includes an overview, column descriptions, and examples to ensure consistency and accuracy in the data extraction process.



We change the current working directory to the specified directory containing the menu images.

This step ensures that all file operations are performed in the correct directory, allowing the script to access the menu images and save the Excel file in the desired location.





In [49]:
# Change the current working directory to the image directory

os.chdir(ROOT_DIR)
IMAGE_DIR = os.path.join(ROOT_DIR,'Dim Sum')

def encode_image(image_path):
    # Open the image file in binary mode and encode it in Base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Process imaged in the directory
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
image_files
test =image_files[0]

Encoding images in Base64 allows us to include image data directly in our API requests without relying on external URLs.

This code scans the directory for files ending with `.png`, `.jpg`, or `.jpeg`, ensuring we only process image files relevant to our task.

We prompt the user to input a name for the new Excel file where the extracted data will be saved.

We loop through each image file, encode it, send it to the OpenAI API for processing, and parse the response to populate our DataFrame.

In [52]:
#Retrieve and encode the image

image_path = os.path.join(IMAGE_DIR,test)
image_data = encode_image(image_path)


In [87]:

# Use GPT-4o to analyze and conver the image

user_content = [
    {"type":"text", "text":"Convert this menu image to a structured Excel Sheet Format."},
    {"type":"image_url", "image_url":{'url':f'data:image/png;base64,{image_data}'}}
]


response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":user_content}
    ],
    temperature=0
)


In [88]:
print(response.choices[0].message.content.split('\n'))
for row in response.choices[0].message.content.split('\n'):
  print(row)

["Here's the structured Excel sheet format based on the menu image:", '', '| Category TitlePT | Category TitleEN | Subcategory TitlePT | Subcategory TitleEN | Item NamePT                  | Item NameEN                  | Item Price | Calories | Portion Size | Availability | Item DescriptionPT                                      | Item DescriptionEN                                      |', '|------------------|------------------|---------------------|---------------------|------------------------------|------------------------------|------------|----------|--------------|--------------|--------------------------------------------------------|--------------------------------------------------------|', '| SOPAS            | Soups            |                     |                     | Sopa Won Ton                 | Won Ton Soup                 | 3,95       |          | 1 unid       |              | Paksoy, Porco e Camarão                                 | Paksoy, Pork & Shrimp          

In [89]:

df = pd.DataFrame()
# Clean and filter only relevant lines (non-separator lines)
lines = [line.strip('|').strip() for line in response.choices[0].message.content.split('\n') if line.startswith('|') and not line.startswith('|-')]

# Join cleaned lines into CSV-style content
cleaned_data = "\n".join(lines)

# Parse into DataFrame
df = pd.read_csv(StringIO(cleaned_data), sep="|")

# Strip whitespace
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [90]:
df

,Category TitlePT,Category TitleEN,Subcategory TitlePT,Subcategory TitleEN,Item NamePT,Item NameEN,Item Price,Calories,Portion Size,Availability,Item DescriptionPT,Item DescriptionEN
0,SOPAS,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1 unid,,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,SOPAS,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1 unid,,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,ARROZ & VERDURAS,Rice & Vegetables,,,Arroz branco,White Rice,"1,95",,1 unid,,Arroz ao vapor,Steamed Rice
3,ARROZ & VERDURAS,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1 unid,,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,SIAO LONG PAO,Siao Long Pao,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,3 unid,,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"
5,SIAO LONG PAO,Siao Long Pao,,,Siao Long Pao Negro,Black Siao Long Pao,"4,50",,3 unid,,"Porco, Gengibre e Cebolinho","Pork, Ginger & Chives"
6,GYOZAS,Gyozas,,,Gyoza Vegetais,Vegetable Gyoza,"4,50",,3 unid,,,NaN
7,GYOZAS,Gyozas,,,Gyoza Camarão,Shrimp Gyoza,"4,50",,3 unid,,,NaN
8,GYOZAS,Gyozas,,,Gyoza Pato,Duck Gyoza,"4,50",,3 unid,,,NaN
9,GYOZAS,Gyozas,,,Gyoza Galinha,Chicken Gyoza,"4,50",,3 unid,,,NaN


#### Loop Through All the Images


In [92]:
df = pd.DataFrame()
header_added = False

for image_file in image_files:
    image_path = os.path.join(IMAGE_DIR, image_file)
    image_data = encode_image(image_path)

    user_content = [
        {"type": "text", "text": "Convert this menu image to a structured Excel Sheet Format."},
        {"type": "image_url", "image_url": {'url': f'data:image/png;base64,{image_data}'}}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}
        ],
        temperature=0
    )

    # Parse the markdown table into CSV-style content
    lines = [
        line.strip('|').strip()
        for line in response.choices[0].message.content.split('\n')
        if line.startswith('|') and not line.startswith('|-')
    ]
    cleaned_data = "\n".join(lines)

    # Convert to DataFrame
    df_new = pd.read_csv(StringIO(cleaned_data), sep='|')

    # Clean columns and values
    df_new.columns = df_new.columns.str.strip()
    df_new = df_new.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Append
    if not header_added:
        df = pd.concat([df, df_new], ignore_index=True)
        header_added = True
    else:
        df = pd.concat([df, df_new.iloc[1:]], ignore_index=True)  # Skip duplicated header row

# Preview result
print(df.head())


   Category TitlePT   Category TitleEN Subcategory TitlePT  \
0             SOPAS              Soups                       
1             SOPAS              Soups                       
2  ARROZ & VERDURAS  Rice & Vegetables                       
3  ARROZ & VERDURAS  Rice & Vegetables                       
4     SIAO LONG PAO                                          

  Subcategory TitleEN                Item NamePT                Item NameEN  \
0                                   Sopa Won Ton               Won Ton Soup   
1                               Sopa Vegetariana            Vegetarian Soup   
2                                   Arroz branco                 White Rice   
3                                Paksoy ao vapor             Steamed Paksoy   
4                      Siao Long Pao Tradicional  Traditional Siao Long Pao   

  Item Price Calories Portion Size Availability  \
0       3,95                1 unid            1   
1       3,95                1 unid            1   

In [93]:
df

,Category TitlePT,Category TitleEN,Subcategory TitlePT,Subcategory TitleEN,Item NamePT,Item NameEN,Item Price,Calories,Portion Size,Availability,Item DescriptionPT,Item DescriptionEN
0,SOPAS,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1 unid,1,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,SOPAS,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1 unid,1,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,ARROZ & VERDURAS,Rice & Vegetables,,,Arroz branco,White Rice,"1,95",,1 unid,1,Arroz ao vapor,Steamed Rice
3,ARROZ & VERDURAS,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1 unid,1,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,SIAO LONG PAO,,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,1 unid,1,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"
5,SIAO LONG PAO,,,,Siao Long Pao Negro,Black Siao Long Pao,"4,50",,1 unid,1,"Porco, Gengibre e Cebolinho","Pork, Ginger & Chives"
6,GYOZAS,,,,Gyoza Vegetais,Vegetable Gyoza,"4,50",,3 unid,1,,NaN
7,GYOZAS,,,,Gyoza Camarão,Shrimp Gyoza,"4,50",,3 unid,1,,NaN
8,GYOZAS,,,,Gyoza Pato,Duck Gyoza,"4,50",,3 unid,1,,NaN
9,GYOZAS,,,,Gyoza Galinha,Chicken Gyoza,"4,50",,3 unid,1,,NaN


In this loop:

* **Encoding the Image**: Each image is encoded in Base64 format using the `encode_image` function.
* **API Request**: We send the encoded image along with the prompt to the OpenAI API using `client.chat.completions.create`.
* **Temperature Parameter**: We set `temperature=0` to make the output deterministic, ensuring consistent formatting.
* **Response Parsing**: The API response is expected to be in a Markdown table format. We parse each line, checking if it's a data row.
* **Data Extraction**: We extract the columns, check if they match the expected number of DataFrame columns, and append them to the DataFrame.


In [94]:
# Save Data Frame to Excel Sheet
path_to_save = os.path.join(IMAGE_DIR,'','menu_data.xlsx')
df.to_excel(path_to_save, index=False)

After processing all images, we save the populated DataFrame to an Excel file.

This script demonstrates how to automate the extraction of structured data from menu images using the OpenAI GPT model.

By converting menu images into a standardized Excel format, we facilitate easier data management and analysis for restaurant or cafe menus.

The use of the OpenAI API for image-to-text conversion streamlines the data entry process, reducing manual effort and potential errors.